<a href="https://colab.research.google.com/github/JSJeong-me/AutoGen/blob/main/ver049/StructuredOutput03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U "autogen-agentchat" "autogen-ext[openai]"

In [ ]:
from google.colab import userdata
import openai
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
openai.api_key  = os.environ["OPENAI_API_KEY"]

In [ ]:
from autogen_core.tools import FunctionTool


# Define a tool using a Python function.
async def web_search_func(query: str) -> str:
    """Find information on the web"""
    return "AutoGen is a programming framework for building multi-agent applications."


# This step is automatically performed inside the AssistantAgent if the tool is a Python function.
web_search_function_tool = FunctionTool(web_search_func, description="Find information on the web")
# The schema is provided to the model during AssistantAgent's on_messages call.
web_search_function_tool.schema


In [16]:
from typing import Literal

from pydantic import BaseModel
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console


# The response format for the agent as a Pydantic base model.
class AgentResponse(BaseModel):
    thoughts: str
    response: Literal["happy", "sad", "neutral"]


# Create an agent that uses the OpenAI GPT-4o model with the custom response format.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    response_format=AgentResponse,  # type: ignore
)
agent = AssistantAgent(
    "assistant",
    model_client=model_client,
    system_message="Categorize the input as happy, sad, or neutral following the JSON format.",
)

await Console(agent.run_stream(task="I am happy."))


---------- user ----------
I am happy.
---------- assistant ----------
{"thoughts":"The statement \"I am happy\" directly expresses the user's emotional state, indicating a positive and pleasant mood. The use of \"happy\" clearly denotes joy and contentment.","response":"happy"}


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='I am happy.', type='TextMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=89, completion_tokens=43), metadata={}, content='{"thoughts":"The statement \\"I am happy\\" directly expresses the user\'s emotional state, indicating a positive and pleasant mood. The use of \\"happy\\" clearly denotes joy and contentment.","response":"happy"}', type='TextMessage')], stop_reason=None)